# Modeling and Simuation Final Project
## DATA604 - Summer 2020
### Completed by: Philip Tanofsky
### Date: June 28, 2020


In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

from pandas import read_html

In [2]:
def plot_results(input1, input2, timeseries, title):
    """Plot the estimates and the mode.
    
    input1:
    input2:
    timeseries:
    title: string
    """
    plot(input1, ':', label='INPUT 1')
    plot(input2, '--', label='INPUT 2')
    plot(timeseries, color='gray', label='model')
    
    decorate(xlabel='Minute',
             ylabel='Score',
             title=title)

In [4]:
def run_simulation(system, update_func):
    """Simulate the system using any update function.
    
    system: System object
    update_func: function that computes the score for each team
    
    returns: TimeSeries
    """
    
    results = TimeSeries()
    results[system.t_0] = system.p_0
    
    for t in linrange(system.t_0, system.t_end):
        results[t+1] = update_func(results[t], t, system)
        
    return results

In [5]:
def update_func_v0(pop, t, system):
    """Compute the score for the team given 1 additional possession.
    
    pop: current population
    t: current year
    system: system object containing parameters of the model
    
    return: population next year
    """
    net_growth = system.alpha * pop + system.beta + pop**2
    return pop + net_growth

In [10]:
# Create a system object
# System is static elements of the system
system = System(team_A=0,
                team_B=0,
                sections=4,
                t_per_section=12)

# Do I need a system for each team? As the percentages are static for each team ... I think Yes.
# Then a state to represent the game for one team, as the score changes
# Also, for the state could track the game stats for the simulation
# Then a larger system for the state/system for each team

,values
team_A,0
team_B,0
sections,4
t_per_section,12


### Thinks to account for
- Overtime games, how to add 5 more minutes
- System object
-- Contains each team object
- State object for each team
- Be able to plot the scores for a game
- Be able to output the results of a X-game series
- Be able to output the results of a simulated tournament

### Steps
- First: Simulate the score for 1 team over the course of 1 NBA game
-- 48 minutes
-- Possessions per minute (or perhaps per quarter)
- Second: Model two teams to simulate a head-to-head game

## Considering Spurs
Average 104 possessions per game
Game 1 of 2019-20 season:
- FGA: 83
- Turnovers: 21

In [2]:
"""
Definition of Possessions: 
https://www.basketball-reference.com/about/glossary.html#:~:text=Poss%20%2D%20Possessions%20(available%20since%20the,FG)%20%2B%20Opp%20TOV)).

Poss - Possessions
(available since the 1973-74 season in the NBA)
the formula for teams is:
    0.5 * 
    (
        (Team_FieldGoalAttempts 
         + 0.4  * Team_FreeThrowAttempts 
         - 1.07 * (   Team_OffensiveRebounds / 
                     (Team_OffensiveRebound  + Opp_DefensiveRebound)
                  )
                * (Team_FieldGoalAttempts - Team_FieldGoals) + Team_Turnover
        ) 
        +
        (Opp_FieldGoalAttempts
         + 0.4 * Opp_FreeThrowAttempts 
         - 1.07 * (    Opp_OffensiveRebounds / 
                      (Opp_OffensiveRebounds + Team_DefensiveRebounds)
                  )
                * (Opp_FieldGoalAttempts - Opp_FieldGoals) + Opp_Turnover
        )
    )
"""

'\nDefinition of Possessions\n'

In [5]:
SA_Poss = 0.5 * ((42.0 + 0.4  * 22.8 - 1.07 * (8.8 / (8.8  + 36.3)) * (89.5 - 42.0) + 12.3) + (89.9 + 0.4 * 22.7 - 1.07 * (9.4 / (9.4 + 35.6)) * (89.9 - 42.2) + 13.0))
SA_Poss

77.41072341463415

In [8]:
# so for a team, a decent estimate of possession per game:
# FieldGoalAttempts + (0.44 * FreeThrowAttempts) + Turnovers - OffensiveRebounds
# Below calculates for SA in season 2019-20 (currently stopped due to Covid-19)
SA_Poss_Simple = 89.5 + (0.44 * 22.8) + 12.3 - 8.8
SA_Poss_Simple

103.032

According to this site: https://www.teamrankings.com/nba/stat/possessions-per-game
SA averages 104.0 possessions per game (close enough for me to 103 above)

This site: http://stats.inpredictable.com/nba/ssnTeamPoss.php
indicates SA scores 1.1 points per possession

103 * 1.1 = 113.3
104 * 1.1 = 114.4

SA averages 113.2 pts per game ... aha the little formula works

so now, per possession, what are the percentages of
2P-FGA
3P-FGA
FT
Turnover

2FGA = 60.8
3FGA = 28.7
FT = 22.8
Turnovers = 12.3
OffensiveRebound = 8.8

San Antonio will have 103 possessions per game, for each possession what is the outcome
2P FGA
3P FGA
FT
TO

What is the probability of any of those outcomes


In [12]:
SA_Poss_Simple = 89.5 + (0.44 * 22.8) + 12.3 - 8.8
SA_Poss_Simple

103.032

In [33]:
SA_Poss_Simple = (60.8 * .517 * 2) + (28.7 * .371 * 3) + (22.8 * .809) + ((12.3 + 8.8)*0)
SA_Poss_Simple

113.2555

In [13]:
60.8 / 89.5

0.6793296089385474

In [ ]:
60.8

In [14]:
28.7 / 89.5

0.3206703910614525

In [15]:
.44 * 22.8

10.032

In [37]:
# I need to subtract the appropriate amount from 8.8 from 2FGA, 3FGA, (exlude FT)
89.5

89.5

In [38]:
60.8 - (60.8 / 89.5 * 8.8)

54.821899441340776

In [39]:
28.7 - (28.7 / 89.5 * 8.8)

25.878100558659217

In [19]:
10 - (10 / 99.5 * 8.8)

9.115577889447236

San Antonio will have 103 possessions per game, for each possession what is the outcome
2P FGA = 54.821899441340776
3P FGA = 25.878100558659217
FT = 10.032
TO = 12.3

In [41]:
54.821899441340776 + 25.878100558659217 + 10 + 12.3

102.99999999999999

In [42]:
54.821899441340776 / 103.0

0.5322514508868037

In [43]:
25.878100558659217 / 103.0

0.2512436947442642

In [44]:
10 / 103.0

0.0970873786407767

In [25]:
12.3 / 103.0

0.11941747572815535

This means, per possession, the percentage of each result are the following:
2P FGA: 0.5322514508868037
3P FGA: 0.2512436947442642
FTAs: 0.0970873786407767
TO: .11941747572815535

In [50]:
# Percentage of possession result with points given the outcome
possession = (0.5322514508868037 * .517 * 2) + (0.2512436947442642 * .371 * 3) + (0.0970873786407767 * .809 * 2) + (.11941747572815535 * 0)
possession

1.0656133004284862

In [51]:
1.1 - possession

0.03438669957151386

In [52]:
possession * 103

109.75816994413408

In [31]:
possession = (.55422713567839196 * .517 * 2) + (.26161708542713566 * .371 * 3) + (.09115577889447236 * .809 * 2) + (.11941747572815535 * 0)
possession

1.0117407246231156

In [32]:
possession * 103

104.2092946361809

In [47]:
0.5322514508868037 + 0.2512436947442642 + 0.0970873786407767 + .11941747572815535

0.9999999999999999

In [ ]:
# even though I don't like the double counting of the offensive rebounds
SA_Poss_Simple = (60.8 * .517 * 2) + (28.7 * .371 * 3) + (22.8 * .809) + ((12.3 + 8.8)*0)

- Made Field Goal Attempts ()
- Made Final Free Throw Attempt
-- “and-One” situations following a converted field goal attempt
-- Final attempt on non-“and-one” attempts.
- Missed Final Free Throw Attempt that results in a Defensive Rebound
- Missed Field Goal Attempt that results in a Defensive Rebound (have count)
- Turnover (have count)
- End of time period (ignore)

okay, so to add more to the equation, if field goal is made, then SA and-one percentage is 4.2
season for fouled on 3-pt attempts is 1.6